# Groupby operations

Some imports:

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

try:
    import seaborn
except ImportError:
    pass

pd.options.display.max_rows = 10

## Recap: the groupby operation (split-apply-combine)

El concepto de "group by": queremos **aplicar la misma función en subconjuntos de su marco de datos, en función de alguna clave para dividir el marco de datos en subconjuntos**

Esta operación también se conoce como la operación "dividir-aplicar-combinar", que incluye los siguientes pasos:

* **Splitting** los datos en grupos según algunos criterios
* **Applying** una función a cada grupo de forma independiente
* **Combining** los resultados en una estructura de datos

<img src="img/splitApplyCombine.png">

Similar to SQL `GROUP BY`

El ejemplo de la imagen en sintaxis de pandas:

In [ ]:
df = pd.DataFrame({'key':['A','B','C','A','B','C','A','B','C'],
                   'data': [0, 5, 10, 5, 10, 15, 10, 15, 20]})
df

Usando las operaciones de filtrado y reducciones que hemos visto en los cuadernos anteriores, podríamos hacer algo como

    df[df['key'] == "A"].sum()
    df[df['key'] == "B"].sum()
    ...


Pero pandas proporciona el método `groupby` para hacer esto:

In [ ]:
df.groupby('key').aggregate('sum')  # np.sum

In [ ]:
df.groupby('key').sum()

Pandas no solo te permite agrupar por un nombre de columna. En `df.groupby (agrupador)` puede haber muchas cosas:

- Serie (o cadena que indica una columna en df)
- función (para aplicar en el índice)
- dict: agrupa por valores
- niveles = [], nombres de niveles en un MultiIndex

In [ ]:
df.groupby(lambda x: x % 2).mean()

## And now applying this on some real data


Estos ejercicios se basan en el tutorial PyCon de Brandon Rhodes (¡así que todo el mérito es para él!) Y los conjuntos de datos que preparó para eso. Puede descargar estos datos desde aquí: títulos.csv y cast.csv y ponerlos en la carpeta / data.

Conjunto de datos `cast`: diferentes roles desempeñados por actores / actrices en películas

- título: título de la película
- nombre: nombre del actor / actriz
- tipo: actor / actriz
- n: el orden del papel (n = 1: papel principal)

In [ ]:
cast = pd.read_csv('data/cast.csv')
cast.head()

In [ ]:
titles = pd.read_csv('data/titles.csv')
titles.head()

<div class="alert alert-success">
    <b>EXERCISE</b>: Usando groupby(), trazar el número de películas que se han estrenado cada década en la historia del cine.
</div>

In [ ]:
titles.groupby(titles.year // 10 * 10).size().plot(kind='bar')

<div class="alert alert-success">
    <b>EXERCISE</b>: Usando groupby() para trazar el número de películas de "Hamlet" realizadas cada década
</div>

In [ ]:
hamlet = titles[titles['title'] == 'Hamlet']
hamlet.groupby(hamlet.year // 10 * 10).size().plot(kind='bar')

<div class="alert alert-success">
    <b>EXERCISE</b>: ¿Cuántos papeles principales (n = 1) estaban disponibles para los actores y cuántos para las actrices en cada año de la década de 1950?
</div>

In [ ]:
cast1950 = cast[cast.year // 10 == 195]
cast1950 = cast1950[cast1950.n == 1]
cast1950.groupby(['year', 'type']).size()

<div class="alert alert-success">
    <b>EXERCISE</b>: Enumere los 10 actores / actrices que tienen la mayor cantidad de papeles principales (n = 1) desde la década de 1990.
</div>

In [ ]:
cast1990 = cast[cast['year'] >= 1990]
cast1990 = cast1990[cast1990.n == 1]
cast1990.groupby('name').size().nlargest(10)

<div class="alert alert-success">
    <b>EXERCISE</b>: Usando groupby() para determinar cuántos roles se enumeran para cada una de las películas de La Pantera Rosa.
</div>

In [ ]:
c = cast
c = c[c.title == 'The Pink Panther']
c = c.groupby(['year'])[['n']].max()
c

<div class="alert alert-success">
    <b>EXERCISE</b>: Enumere, en orden por año, cada una de las películas en las que Frank Oz ha interpretado más de un papel.
</div>

In [ ]:
c = cast
c = c[c.name == 'Frank Oz']
g = c.groupby(['year', 'title']).size()
g[g > 1]

<div class="alert alert-success">
    <b>EXERCISE</b>: Enumere cada uno de los personajes que Frank Oz ha interpretado al menos dos veces.
</div>

In [ ]:
c = cast
c = c[c.name == 'Frank Oz']
g = c.groupby(['character']).size()
g[g > 1].sort_values()

## Transforms

A veces, no desea agregar los grupos, sino transformar los valores en cada grupo. Esto se puede lograr con `transform`:

In [ ]:
df

In [ ]:
df.groupby('key').transform('mean')

In [ ]:
def normalize(group):
    return (group - group.mean()) / group.std()

In [ ]:
df.groupby('key').transform(normalize)

In [ ]:
df.groupby('key').transform('sum')

<div class="alert alert-success">
    <b>EXERCISE</b>: Agregue una columna al marco de datos `cast` que indique el número de roles para la película.
</div>

In [ ]:
cast['n_total'] = cast.groupby('title')['n'].transform('max')
cast.head()

<div class="alert alert-success">
    <b>EXERCISE</b>: Calcule la proporción de papeles principales de actor y actriz con respecto al número total de papeles principales por década.
</div>

Consejo: puede hacer un grupo dos veces en dos pasos, una vez calculando los números y luego las proporciones.

In [ ]:
leading = cast[cast['n'] == 1]
sums_decade = leading.groupby([cast['year'] // 10 * 10, 'type']).size()
sums_decade

In [ ]:
#sums_decade.groupby(level='year').transform(lambda x: x / x.sum())
ratios_decade = sums_decade / sums_decade.groupby(level='year').transform('sum')
ratios_decade

In [ ]:
ratios_decade[:, 'actor'].plot()
ratios_decade[:, 'actress'].plot()

## Intermezzo: string manipulations

Las cadenas de Python tienen muchos métodos útiles disponibles para manipular o verificar el contenido de la cadena:

In [ ]:
s = 'Bradwurst'

In [ ]:
s.startswith('B')

En pandas, esos métodos (junto con algunos métodos adicionales) también están disponibles para series de cadenas a través del descriptor de acceso `.str`:

In [ ]:
s = pd.Series(['Bradwurst', 'Kartoffelsalat', 'Sauerkraut'])

In [ ]:
s.str.startswith('B')

For an overview of all string methods, see: http://pandas.pydata.org/pandas-docs/stable/api.html#string-handling

<div class="alert alert-success">
    <b>EXERCISE</b>: Ya trazamos el número de películas de 'Hamlet' estrenadas cada década, pero no todos los títulos se llaman exactamente 'Hamlet'. Proporcione una descripción general de los títulos que contienen 'Hamlet' y que comienzan con 'Hamlet':
</div>

In [ ]:
hamlets = titles[titles['title'].str.contains('Hamlet')]
hamlets['title'].value_counts()

In [ ]:
hamlets = titles[titles['title'].str.match('Hamlet')]
hamlets['title'].value_counts()

<div class="alert alert-success">
    <b>EXERCISE</b>: Enumere los 10 títulos de películas con el nombre más largo.
</div>

In [ ]:
title_longest = titles['title'].str.len().nlargest(10)
title_longest

In [ ]:
pd.options.display.max_colwidth = 210
titles.loc[title_longest.index]

## Value counts

Un atajo útil para calcular el número de ocurrencias de ciertos valores es `value_counts` (esto es algo equivalente a` df.groupby (key) .size ()) `)

Por ejemplo, ¿cuáles son los títulos de películas más frecuentes?

In [ ]:
titles.title.value_counts().head()

<div class="alert alert-success">
    <b>EXERCISE</b>: ¿En qué años se estrenaron más películas?
</div>

In [ ]:
t = titles
t.year.value_counts().head(3)

<div class="alert alert-success">
    <b>EXERCISE</b>: Trazar el número de películas lanzadas a lo largo del tiempo.
</div>

In [ ]:
titles.year.value_counts().sort_index().plot()

<div class="alert alert-success">
    <b>EXERCISE</b>: Traza el número de películas de "Hamlet" realizadas cada década.
</div>

In [ ]:
t = titles
t = t[t.title == 'Hamlet']
(t.year // 10 * 10).value_counts().sort_index().plot(kind='bar')

<div class="alert alert-success">
    <b>EXERCISE</b>: ¿Cuáles son los 11 nombres de personajes más comunes en la historia del cine?
</div>

In [ ]:
cast.character.value_counts().head(11)

<div class="alert alert-success">
    <b>EXERCISE</b>: ¿Qué actores o actrices aparecieron en más películas en el año 2010?
</div>

In [ ]:
cast[cast.year == 2010].name.value_counts().head(10)

<div class="alert alert-success">
    <b>EXERCISE</b>: Trace la cantidad de papeles que Brad Pitt ha desempeñado en cada año de su carrera.
</div>

In [ ]:
cast[cast.name == 'Brad Pitt'].year.value_counts().sort_index().plot()

<div class="alert alert-success">
    <b>EXERCISE</b>: ¿Cuáles son los 10 papeles más títulos de películas que comienzan con la palabra "La vida"?
</div>

In [ ]:
c = cast
c[c.title.str.startswith('The Life')].title.value_counts().head(10)

<div class="alert alert-success">
    <b>EXERCISE</b>: ¿Cuántos papeles principales (n = 1) estaban disponibles para los actores y cuántos para las actrices en la década de 1950? ¿Y en la década de 2000?
</div>

In [ ]:
c = cast
c = c[c.year // 10 == 195]
c = c[c.n == 1]
c.type.value_counts()

In [ ]:
c = cast
c = c[c.year // 10 == 200]
c = c[c.n == 1]
c.type.value_counts()